# Explicamelo como si tuviera 5 años


![Architecture](images/architecture.png)

En este cuaderno vamos a recorrer el proceso de configurar un chatbot simple en LangChain.

A lo largo de este proceso, vamos a mostrar cómo LangSmith puede usarse para mejorar la experiencia del desarrollador en aplicaciones de IA.

Empecemos cargando nuestras variables de entorno desde el archivo .env.

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)
# Cargar las siguientes variables de entorno:
# LANGSMITH_TRACING=true
# LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
# LANGSMITH_PROJECT="eli5-bot"
# LANGSMITH_API_KEY="<redacted>"

# OPENAI_API_KEY="<redacted>"
# TAVILY_API_KEY="<redacted>"

True

## Setup

Configuremos una herramienta llamada Tavily para permitir que nuestro asistente busque en la web al responder.

In [4]:
from langchain_tavily import TavilySearch

web_search_tool = TavilySearch(
    max_results=1
)

Diseñemos un prompt para RAG que usaremos a lo largo de todo el cuaderno.

In [5]:
prompt = """Sos un profesor y un experto en explicar temas complejos de una manera fácil de entender.
Tu trabajo es responder la pregunta dada de forma tal que incluso un niño de 5 años pueda comprenderla.
Se te ha proporcionado el contexto necesario para responder la pregunta.

Pregunta: {question} 

Contexto: {context}

Respuesta:"""
print("Prompt Template: ", prompt)

Prompt Template:  Sos un profesor y un experto en explicar temas complejos de una manera fácil de entender.
Tu trabajo es responder la pregunta dada de forma tal que incluso un niño de 5 años pueda comprenderla.
Se te ha proporcionado el contexto necesario para responder la pregunta.

Pregunta: {question} 

Contexto: {context}

Respuesta:


## Creando nuestra aplicación

In [8]:
from openai import OpenAI
from langsmith import traceable
from langsmith.wrappers import wrap_openai

openai_client = wrap_openai(OpenAI())

@traceable
def search(question):
    web_docs = web_search_tool.invoke({"query": question})
    if isinstance(web_docs, str):
        return web_docs
    return "\n".join(
        d if isinstance(d, str)
        else d.get("content", "")
        for d in web_docs
    )

@traceable
def explain(question, context):
    formatted = prompt.format(question=question, context=context)
    
    completion = openai_client.chat.completions.create(
        messages=[
            {"role": "system", "content": formatted},
            {"role": "user", "content": question},
        ],
        model="gpt-4o-mini",
    )
    return completion.choices[0].message.content

@traceable
def eli5(question):
    context = search(question)
    answer = explain(question, context)
    return answer

## Testeando nuestra application

In [9]:
question = "Qué es la macroeconomía?"
print(eli5(question))

La macroeconomía es como mirar a un país desde lejos, como si fueras un pájaro volando por encima. En vez de fijarte en una sola persona o familia, te fijas en todo lo que hacen las personas en ese país. 

Piensa en esto: si todos en un país compran muchas cosas, como juguetes o comida, eso se llama "economía". La macroeconomía estudia eso, cómo le va a todo el mundo, cuántos trabajos hay, y si la gente tiene suficiente dinero para vivir bien. 

Así que, en resumen, la macroeconomía es lo que pasa en un país completo acerca de todo el dinero y las cosas que compramos y vendemos. ¡Es como contar cuántos globos de diferentes colores hay en una fiesta, pero en lugar de globos, se trata de dinero y cosas que la gente usa!
